<a href="https://www.kaggle.com/code/supaseksamurai88/store-sales-james-eda?scriptVersionId=143546632" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# This notebook is a part of CSC345 Data Science. 
In this project phase, we will be looking on all features in this Store Sales competition.

## Introduction

### Q1.What problem does it solve and for whom?

*Answer: Local groceries must identify the potential need of the customer, so the retailer stores itself could prepare the right product with right amount at the right time for those customer.*

### Q2.How is the problem being solved today (if at all)?

*Answer: Currently, they use their own forecasting tools. But it's not automated and not applicable when there are newly added branch at new area.*

### Q3.What are the data sets available for the problem? Where do they come from?

Answers: There are 4 main datasets we will be lookign for our understanding and train our data. The test_data.csv is for submission.
#### 1. train.csv, comprises of these 4 attributes **(3,000,888 rows, 6 columns)**
* **store_nbr** identifies the store at which the products are sold.
* **family** identifies the type of product sold.
* **sales** gives the total sales for a product family at a particular store at a given date. Fractional values are possible since products can be sold in fractional units (1.5 kg of cheese, for instance, as opposed to 1 bag of chips).
* **onpromotion** gives the total number of items in a product family that were being promoted at a store at a given date.

#### 2. stores.csv, contains all information related to that city where the branch is opearted, comprises of 4 attributes. **(54 rows, 5 columns)**
* Store metadata, including **city**, **state**, **type**, and **cluster**.
* **cluster** is a grouping of similar stores.

#### 3. oil.csv,daily oil price.  **(1,218 rows, 2 columns)**
* Includes values during both the train and test data timeframes. (Ecuador is an oil-dependent country and it's economical health is highly vulnerable to shocks in oil prices.)

#### 4. holidays_events.csv, Holidays and Events, with metadata **(350 rows, 6 columns)**
* NOTE: Pay special attention to the **transferred** column. A holiday that is transferred officially falls on that calendar day, but was moved to another date by the government. A transferred day is more like a normal day than a holiday. To find the day that it was actually celebrated, look for the corresponding row where type is Transfer. For example, the holiday Independencia de Guayaquil was transferred from 2012-10-09 to 2012-10-12, which means it was celebrated on 2012-10-12. Days that are type Bridge are extra days that are added to a holiday (e.g., to extend the break across a long weekend). These are frequently made up by the type Work Day which is a day not normally scheduled for work (e.g., Saturday) that is meant to payback the Bridge.
* Additional holidays are days added a regular calendar holiday, for example, as typically happens around Christmas (making Christmas Eve a holiday).


### Q4.How are the results of the problem solution to be exposed?
*Answer: Could be integrated to the exisiting forecasting tools of the company, for now the result of the prediction will be in 2 columns table in csv format file. The first column indicates id and the second indicates sales on that particular type of product on that day of the id. (Reference from the sample_submission.csv file)*

### Q5.What type of problem is this?
*Answer: Revenue leakage, the company tries to save more budget by only focusing on purchasing the products which are really needed at the right amount. If the stores are overly-prepared, they could end up generating lots of waste in case of perishable products and losing free money, while the under-prepared stores could end up having not enough product for the customer.*

*Extract from the Compeitition description, "For grocery stores, more accurate forecasting can decrease food waste related to overstocking and improve customer satisfaction. The results of this ongoing competition, over time, might even ensure your local store has exactly what you need the next time you shop.".*


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

Useful link to help explaining parse_date attribute for read_csv


https://saturncloud.io/blog/how-to-parse-dates-in-different-columns-with-pandas-readcsv/#:~:text=To%20parse%20dates%20in%20different%20columns%20with%20read_csv()%20%2C%20we,parse%20dates%20in%20all%20columns.

In [ ]:
oil_data = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/oil.csv', parse_dates = ['date'])
holidays_data = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv', parse_dates = ['date'])
store_data = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/stores.csv')
train_data = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/train.csv', parse_dates = ['date'])
transaction_data = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/transactions.csv')

# Oil Data
Columns: date, dcoilwtico

In [ ]:

# oil_line = oil_data.set_index('date')
oil_data.head()
# oil_data.shape

In [ ]:
sns.lineplot(y = oil_data.dcoilwtico, x = oil_data.date)

In [ ]:
sns.lineplot(x=oil_data.date, y=oil_data.dcoilwtico, lw=3, label='Oil Price')
sns.lineplot(data=train_data.groupby('date')['sales'].mean().reset_index(), x='date', y='sales', 
             lw=0.6, label='Average Sales')

# Holidays Data
Columns: date, type, locale, locale_name, description, transferred

In [ ]:
holidays_data.head()
holidays_data.shape

# Store Data
Columns: store_nbr, city, state, type, cluster

In [ ]:
# store_data.head()
store_data.shape

In [ ]:
store_data.state.unique()

In [ ]:
store_data.city.unique()

In [ ]:
sns.histplot(store_data['type'])

## Try to merge store data and train data to see how much transaction taken place in each branch

In [ ]:
store_master = pd.merge(train_data, store_data, on='store_nbr', how='left')
store_master.head()

In [ ]:
store_master = store_master.set_index('id')
store_master

In [ ]:
# Set the width and height of the figure
plt.figure(figsize=(14,6))

# Add title
plt.title("Average Daily Sales grouped by Stores Type")


sns.lineplot(data=store_master.loc[store_master.type == 'A'].groupby('date')['sales'].mean().reset_index(), x='date', y='sales', 
             lw=0.2, label='Store type A')
sns.lineplot(data=store_master.loc[store_master.type == 'B'].groupby('date')['sales'].mean().reset_index(), x='date', y='sales', 
             lw=0.2, label='Store type B')
sns.lineplot(data=store_master.loc[store_master.type == 'C'].groupby('date')['sales'].mean().reset_index(), x='date', y='sales', 
             lw=0.2, label='Store type C')
sns.lineplot(data=store_master.loc[store_master.type == 'D'].groupby('date')['sales'].mean().reset_index(), x='date', y='sales', 
             lw=0.2, label='Store type D')
sns.lineplot(data=store_master.loc[store_master.type == 'E'].groupby('date')['sales'].mean().reset_index(), x='date', y='sales', 
             lw=0.2, label='Store type E')

## In which state, stores with type A are mostly operated?

In [ ]:
store_master.head()

In [ ]:
# Add title
plt.title("States with store type A operated")

sns.histplot(store_data['state'].loc[store_data.type == 'A'])

Now let's see which city has type A store?

In [ ]:
# Add title
plt.title("States with store type A operated")

sns.histplot(store_data['city'].loc[store_data.type == 'A'])

# Train data
Columns: id, date, store_nbr, family, sales, onpromotion

I will be transforming date data to month format, so we could see the trend in more understandable way.
https://saturncloud.io/blog/how-to-group-by-month-from-a-date-field-using-pythonpandas/#:~:text=By%20using%20the%20groupby%20method,tz_convert%20to%20handle%20time%20zones.


And here is a source to some interesting tips to deal with time series data.
https://machinelearningmastery.com/time-series-data-visualization-with-python/

In [ ]:
train_data.head()
# train_data.shape
# train_data['family'].unique()

In [ ]:
family_class = ['AUTOMOTIVE', 'BABY CARE', 'BEAUTY', 'BEVERAGES', 'BOOKS',
       'BREAD/BAKERY', 'CELEBRATION', 'CLEANING', 'DAIRY', 'DELI', 'EGGS',
       'FROZEN FOODS', 'GROCERY I', 'GROCERY II', 'HARDWARE',
       'HOME AND KITCHEN I', 'HOME AND KITCHEN II', 'HOME APPLIANCES',
       'HOME CARE', 'LADIESWEAR', 'LAWN AND GARDEN', 'LINGERIE',
       'LIQUOR,WINE,BEER', 'MAGAZINES', 'MEATS', 'PERSONAL CARE',
       'PET SUPPLIES', 'PLAYERS AND ELECTRONICS', 'POULTRY',
       'PREPARED FOODS', 'PRODUCE', 'SCHOOL AND OFFICE SUPPLIES',
       'SEAFOOD']



## Sales Growth over years

In [ ]:
sns.lineplot(x = train_data.date, y = train_data.sales)
plt.xlabel("Date")

In [ ]:
# the top 10 product family by sales

sns.barplot(data=train_data.groupby('family')['sales'].mean().sort_values(ascending=False).reset_index(), 
            x='sales', y='family', color='#2596be')

plt.title('Top 10 Family Product by Average Sales')
plt.ylabel('Family Product')
plt.xlabel('Sales')
plt.show()

## Let's have a look on each possible product family

Start with automotive_family

In [ ]:
automotive_data = train_data.loc[train_data.family == 'AUTOMOTIVE']
automotive_data.set_index('date')
# print(len(family_class))

In [ ]:
# convert the date column to a datetime object
automotive_data['date'] = pd.to_datetime(automotive_data['date'])
automotive_data.head()
# group by month


automotive_line = automotive_data.groupby(pd.Grouper(key='date', freq='M')).sum()
# automotive_line.head()